In [ ]:
!pip3 install -qU timm neptune 

In [ ]:
def print_config(config):
    attrs = {}
    attrs.update(config.__dict__)

    for key, value in config.__class__.__dict__.items():
        if not key.startswith('__') and key not in attrs:
            attrs[key] = value

    for key, value in attrs.items():
        print(f"{key}: {value}")

In [ ]:
import torch


class Config:
    model_name = "mlp.baseline"
    input_dim = 2048
    target_dim = 1

    device = "cuda" if torch.cuda.is_available() else "cpu"
    seed = 5274

    optimizer_name = "AdamW"
    lr = 1e-4
    weight_decay = 1e-4

    batch_size = 32  # // grad_accum_steps

    num_epochs = 100
    scheduler_name = "default"

    dropout = 0.1

    hf_token = "ahf_uOkImkbEroqtIuyvGJrttTzaebfeIdPZID"
    neptune_token = "eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJlOGE2YjNiZS1mZGUyLTRjYjItYTg5Yy1mZWJkZTIzNzE1NmIifQ=="
    with_id = ""
    resume = False

config = Config()

In [ ]:
import random
import numpy as np


def setup_reproducibility(config):
    random.seed(config.seed)
    np.random.seed(config.seed)
    torch.manual_seed(config.seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(config.seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(False, warn_only=True)
    torch.set_float32_matmul_precision("high")

setup_reproducibility(config)

In [ ]:
from huggingface_hub import login, snapshot_download

login(config.hf_token)
repo_id = "ArbaazBeg/crunchdao-structural-break-detection"
path = snapshot_download(repo_id, repo_type="dataset")

In [ ]:
import os

inputs_path = path+"/X_train.parquet"
targets_path = path+"/y_train.parquet"
os.listdir(path)

In [ ]:
import pandas as pd

def load_parquet(path):
    return pd.read_parquet(path)

inputs_df = load_parquet(inputs_path)
targets_df = load_parquet(targets_path)

In [ ]:
from sklearn.model_selection import train_test_split

def split(inputs_df, targets_df, seed):
    targets_df = targets_df.reset_index()

    train_ids, test_ids = train_test_split(
        targets_df['id'],                     
        test_size=0.2,
        random_state=seed,
        stratify=targets_df['structural_breakpoint']
    )

    train_inputs = inputs_df.loc[inputs_df.index.get_level_values('id').isin(train_ids)].copy()
    eval_inputs = inputs_df.loc[inputs_df.index.get_level_values('id').isin(test_ids)].copy()

    train_targets = targets_df[targets_df['id'].isin(train_ids)].set_index('id').copy()
    eval_targets = targets_df[targets_df['id'].isin(test_ids)].set_index('id').copy()
    
    return (
        train_inputs,
        train_targets, 
        eval_inputs,
        eval_targets
    )
    
data = split(inputs_df, targets_df, 1)

In [ ]:
import torch
import torch.nn.functional as F


def preprocess_inputs(inputs_df, dimension=2048):
    inputs = []    
    for id, seq in inputs_df.groupby("id"):
        value = torch.tensor(seq["value"].values).float()
        period = torch.tensor(seq["period"].values).float()
        input = torch.stack([value, period], axis=0).unsqueeze(0)
        input = F.interpolate(input, size=dimension, mode="nearest-exact").squeeze(0)
        inputs.append(input) # 2, SEQ LEN
        
    return torch.stack(inputs)

train_inputs = preprocess_inputs(data[0])
eval_inputs = preprocess_inputs(data[2])
train_inputs.shape, eval_inputs.shape

In [ ]:
def preprocess_targets(target_df):
    targets = []
    for id, target in target_df.groupby("id"):
        target = target["structural_breakpoint"].values.astype(np.float32)
        targets.append(torch.tensor(target))
    return torch.stack(targets)    

train_targets = preprocess_targets(data[1])
eval_targets = preprocess_targets(data[3])
train_targets.shape, eval_targets.shape

In [ ]:
def quantile_and_clamp(train_inputs, eval_inputs):
    x = train_inputs[:, 0]
    p1 = torch.quantile(x, 0.01)
    p99 = torch.quantile(x, 0.99)

    print("1st percentile:", p1.item())
    print("99th percentile:", p99.item())

    x = torch.clamp(x, min=p1.item(), max=p99.item())
    eval_inputs[:, 0] = torch.clamp(eval_inputs[:, 0], min=p1.item(), max=p99.item())
    train_inputs[:, 0] = x
    
    return train_inputs, eval_inputs

def return_stats(tensor, p=True):
    mean, std = tensor.mean(), tensor.std()
    min, max =  tensor.min(), tensor.max()

    if p:
        print(f"Min: {min}, Max: {max}, Mean: {mean}, Std: {std}")
        
    return min, max, mean, std

In [ ]:
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset
import numpy as np

class SequentialDataset(Dataset):
    def __init__(
        self, 
        inputs, 
        targets,
        mean=None,
        std=None,
        min=None,
        max=None,
        minmax=False,
        zscore=False,
    ):  
        inputs = inputs.clone()
        targets = targets.clone()
        
        if zscore:
            inputs = self.perform_zscore(inputs, mean, std)
        if minmax:
            inputs = self.perform_minmax(inputs, min, max)
            
        assert len(inputs) == len(targets), "Length Error"
        self.inputs = inputs
        self.targets = targets
            
    def perform_zscore(self, inputs, mean, std):
        for i in range(len(inputs)):
            inputs[i][0] = (inputs[i][0] - mean) / std
        return inputs

    def perform_minmax(self, inputs, min, max):
        for i in range(len(inputs)):
            inputs[i][0] = (inputs[i][0] - min) / (max - min + 1e-8)
        return inputs
    
    def __len__(self):
        return len(self.inputs)
    
    def __getitem__(self, index):   
        return self.inputs[index], self.targets[index]

train_inputs, eval_inputs = quantile_and_clamp(train_inputs, eval_inputs)
min, max, mean, std = return_stats(train_inputs[:, 0])

train_ds = SequentialDataset(
    inputs=train_inputs, 
    targets=train_targets, 
    mean=mean, 
    std=std,
    min=min,
    max=max,
    zscore=True,
    minmax=False
)

eval_ds = SequentialDataset(
    inputs=eval_inputs, 
    targets=eval_targets, 
    mean=mean, 
    std=std,
    min=min,
    max=max,
    zscore=True,
    minmax=False
)

In [ ]:
from torch.utils.data import DataLoader


def build_loader(
    SEED,
    ds,
    train=True,
    batch_size=1,
    shuffle=False,
    num_workers=4,
    drop_last=True,
    pin_memory=True,
    persistent_workers=False,
):
    def seed_worker(worker_id):
        worker_seed = torch.initial_seed() % 2**32
        np.random.seed(worker_seed)
        random.seed(worker_seed)

    generator = torch.Generator()
    generator.manual_seed(SEED if train else SEED+1)

    return DataLoader(
        ds,
        batch_size=batch_size,
        shuffle=shuffle,
        num_workers=num_workers,
        pin_memory=pin_memory,
        drop_last=drop_last,
        persistent_workers=persistent_workers,
        worker_init_fn=seed_worker,
        generator=generator,
        #sampler=DistributedSampler(
        #    train_ds,
        #    shuffle=True,
        #    drop_last=True,
        #    seed=config.seed
        #)
    )
    
    
train_dl = build_loader(
    config.seed,
    train_ds,
    train=True,
    batch_size=config.batch_size,
    shuffle=True,
    num_workers=0,
    drop_last=True,
    pin_memory=True,
    persistent_workers=False,
)

eval_dl = build_loader(
    config.seed,
    eval_ds,
    train=False,
    batch_size=config.batch_size,
    shuffle=False,
    num_workers=0,
    drop_last=True,
    pin_memory=True,
    persistent_workers=False,
)

In [ ]:
import torch.nn as nn


class MLPBlock(nn.Module):
    def __init__(
        self,
        in_features,
        hidden_features,
        dropout,
    ):
        super().__init__()
        self.linear1 = nn.Linear(in_features, hidden_features)
        self.act = nn.ReLU(inplace=True)
        self.drop1 = nn.Dropout(dropout)
        self.norm = nn.BatchNorm1d(2)
        self.linear2 = nn.Linear(hidden_features, hidden_features)
        self.drop2 = nn.Dropout(dropout)
        
    def forward(self, x):
        B = x.shape[0]
        x = self.linear1(x)
        x = self.act(x)
        x = self.drop1(x)
        #x = x.view(B, 40, 2)
        x = self.norm(x)
        #x = x.view(B, 2, 40)
        x = self.linear2(x)
        x = self.drop2(x)
        return x

In [ ]:
class Model(nn.Module):
    def __init__(
        self, 
        input_features,
        hidden_features, 
        dropout=0.1,
        num_blks=1
    ):
        super().__init__()
        blks = []
        for i in range(num_blks):
            blks.append(
                MLPBlock(
                    in_features=input_features,
                    hidden_features=hidden_features,
                    dropout=dropout
                )
            )
            input_features = hidden_features
        self.blks = nn.Sequential(*blks)
        self.final = nn.Linear(hidden_features, 1)

    def forward(self, x):
        x = self.blks(x)
        x = x.mean(dim=1)
        x = self.final(x)
        return x
        

model = Model(
    input_features=2048,
    hidden_features=4096, 
    dropout=config.dropout,
    num_blks=1
).to(config.device)

optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=config.lr,
    weight_decay=config.weight_decay,
    fused=True
)

sum(p.numel() for p in model.parameters()) / 1e6

In [ ]:
from transformers import get_cosine_schedule_with_warmup


total_training_steps = (len(train_dl)) * config.num_epochs
warmup_steps = int(total_training_steps * 0.05)

scheduler = get_cosine_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=total_training_steps
)

In [ ]:
from collections import OrderedDict


def remove_orig_mod(state_dict):
    new_state_dict = OrderedDict()
    prefix = "_orig_mod."
    for key, value in state_dict.items():
        if key.startswith(prefix):
            new_key = key[len(prefix):]
        else:
            new_key = key
        new_state_dict[new_key] = value
    return new_state_dict

In [ ]:
import neptune

def setup_neptune(config):
    if not config.resume:
        neptune_run = neptune.init_run(
            project="arbaaz/bitgrit-air-pollution-1",
            name=config.model_name,
            api_token=config.neptune_token
        )

        neptune_run["h_parameters"] = {
            "model_name": config.model_name,
            "optimizer_name": config.optimizer_name,
            "learning_rate": config.lr,
            "scheduler_name": config.scheduler_name,
            "weight_decay": config.weight_decay,
            "dropout": config.dropout,
            "num_epochs": config.num_epochs,
            "batch_size": config.batch_size,
        }
    else:
        neptune_run = neptune.init_run(
            project="arbaaz/onward-speed-and-structure",
            with_id=config.with_id,
            api_token=config.neptune_token
        )

    return neptune_run

In [ ]:
from sklearn.metrics import roc_auc_score


def loss_fn(logits, targets):
    return F.binary_cross_entropy_with_logits(logits, targets)


def metric_fn(preds, targets):
    preds = preds.sigmoid().squeeze()
    preds = (preds > 0.5).long()
    targets = targets.squeeze().long()
    return roc_auc_score(targets.numpy(), preds.numpy())

In [ ]:
from tqdm.auto import tqdm
            

def train_step(model, inputs, targets):
    with torch.autocast(device_type=config.device, dtype=torch.float16, cache_enabled=True):
        logits = model(inputs)
        loss = loss_fn(logits, targets)
    return loss, logits, targets


def eval_step(model, inputs, targets):
    with torch.inference_mode():
        with torch.autocast(device_type=config.device, dtype=torch.float16, cache_enabled=True):
            logits = model(inputs)
            loss = loss_fn(logits, targets)
    return loss, logits, targets


def inference_step(model, inputs):
    model.eval() # FIX THIS 
    with torch.inference_mode():
        with torch.autocast(device_type=config.device, dtype=torch.float16, cache_enabled=True):
            logits = model(inputs)
    return logits
    

def backward_step(loss, optimizer, scaler, scheduler):
    scaler.scale(loss).backward()
    scaler.step(optimizer)
    scaler.update()
    optimizer.zero_grad()
    scheduler.step()
          
    
def reduce_outputs(outputs):
    data, keys = {}, []
    for key in outputs[0].keys():
        if outputs[0][key].dim() == 0 and ("logits" not in key or "targets" not in key):
            value = torch.stack([x[key] for x in outputs]).mean().item()
            data[key] = value
            keys.append(key)
            
    for i in range(len(outputs)): # delete the scalar and loss keys 
        for k in keys:
            del outputs[i][k]
    
    o = reduce_dimensional_outputs(outputs)
    return data | o


def reduce_dimensional_outputs(outputs):
    o = {}
    for key in outputs[0].keys():
        if "logits" in key or "targets" in key:
            tensor = torch.stack([x[key] for x in outputs]).flatten()
            o[key] = tensor
    return o


def save_ckpt(model, metadata, ckpt_path):
    data = {"state_dict": model.state_dict()}
    for k, v in metadata.items():
        data[k] = v
    torch.save(data, ckpt_path)


def log_and_print(neptune_run, data, log=True, print=True):
    if log:
        for key, value in data.items():
            neptune_run[key].append(value)
        
    if print:
        tqdm.write((
            f"Epoch: {data['epoch']}, "
            f"train/loss: {data['train/loss']:.4f}, "
            f"eval/loss: {data['eval/loss']:.4f}, "
            f"train/auroc: {data['train/auroc']:.4f}, "
            f"eval/auroc: {data['eval/auroc']:.4f}, "
        ))

In [25]:
def train_one_epoch(
    model, 
    optimizer,
    scaler,  
    scheduler,
    train_dl,
    epoch,
    num_epochs,
    resume_epoch,
    resume, 
    log_lr_on_step,
    neptune_run
):
    model.train()
    pbar = tqdm(
        train_dl,
        desc=f"Epoch: {epoch}/{num_epochs}",
        leave=False,
        dynamic_ncols=True,
        colour="red",
        position=1,
    )
    outputs = []

    for index, (inputs, targets) in enumerate(pbar):
        if resume and epoch < resume_epoch:
            continue
        
        inputs = inputs.to(config.device, non_blocking=True)
        targets = targets.to(config.device, non_blocking=True)
    
        loss, logits, targets = train_step(model, inputs, targets)
        backward_step(loss, optimizer, scaler, scheduler)

        outputs.append({
            "train/loss": loss.detach().cpu(),
            "train/logits": logits.detach().cpu(),
            "train/targets": targets.detach().cpu()
        })

        if log_lr_on_step:
            log_and_print(
                neptune_run,
                data={"STEP/lr": scheduler.get_last_lr()[0]}, 
                log=True, 
                print=False
            )
            
    return outputs
            
            
def evaluate(model, eval_dl):
    model.eval()
    pbar = tqdm(
        eval_dl,
        desc=f"Evaluating",
        leave=False,
        dynamic_ncols=True,
        colour="blue",
        position=1,
    )
    outputs = []
    
    for inputs, targets in pbar:
        inputs = inputs.to(config.device, non_blocking=True)
        targets = targets.to(config.device, non_blocking=True)
        loss, logits, targets = eval_step(model, inputs, targets)
        
        outputs.append({
            "eval/loss": loss.detach().cpu(),
            "eval/logits": logits.detach().cpu(),
            "eval/targets": targets.detach().cpu()
        })
        
    return outputs

In [ ]:
import traceback


def train(
    model, 
    optimizer, 
    scheduler, 
    train_dl,
    eval_dl, 
    num_epochs, 
    resume_epoch,
    resume,
    log_lr_on_step,
    neptune_run
):
    os.makedirs("/ckpts", exist_ok=True)
    train_outputs = []
    eval_outputs = []
    score = float('-inf')
    
    scaler = torch.GradScaler()
    pbar = tqdm(range(0, num_epochs), position=0, colour="green")
    
    for epoch in pbar:
        train_outputs = train_one_epoch(
            model,
            optimizer, 
            scaler, 
            scheduler, 
            train_dl,
            epoch,
            num_epochs,
            resume_epoch,
            resume,
            log_lr_on_step,
            neptune_run
        )
        eval_outputs = evaluate(model, eval_dl)
        
        train_outputs = reduce_outputs(train_outputs)
        eval_outputs = reduce_outputs(eval_outputs)
        train_outputs["train/auroc"] = metric_fn(train_outputs["train/logits"], train_outputs["train/targets"])
        eval_outputs["eval/auroc"] = metric_fn(eval_outputs["eval/logits"], eval_outputs["eval/targets"])
        
        outputs = train_outputs | eval_outputs
        outputs["epoch"] = epoch
        del outputs["train/logits"]
        del outputs["eval/logits"]
        del outputs["train/targets"]
        del outputs["eval/targets"]
        
        log_and_print(neptune_run, outputs)

        if outputs["eval/auroc"] > score:
            score = outputs["eval/auroc"]
            metadata = {
                "train_auroc": outputs["train/auroc"],
                "eval_auroc": score,
                "epoch": epoch
            }
            save_ckpt(model, metadata, f"/ckpts/{config.model_name}.pt")
            
        #if RMSE >= torch.tensor(0.7000).cuda():
        #    break
            
    neptune_run.stop()
    
    
def start_training():
    try:
        neptune_run = setup_neptune(config)
        train(
            model, 
            optimizer, 
            scheduler, 
            train_dl,
            eval_dl, 
            num_epochs=config.num_epochs, 
            resume_epoch=0,
            resume=config.resume,
            log_lr_on_step=True,
            neptune_run=neptune_run
        )
    except Exception as e:
        print(f"Exception occurred: {e}")
        traceback.print_exc()
    finally:
        pass
        neptune_run.stop()
        
start_training()

[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/arbaaz/bitgrit-air-pollution-1/e/BIT-118


  0%|          | 0/100 [00:00<?, ?it/s]

Epoch: 0/100:   0%|          | 0/250 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/62 [00:00<?, ?it/s]

eval/loss
eval/logits
eval/targets
Epoch: 0, train/loss: 0.3847, eval/loss: 0.9667, train/auroc: 0.7330, eval/auroc: 0.4851, 


...

Evaluating:   0%|          | 0/62 [00:00<?, ?it/s]

eval/loss
eval/logits
eval/targets
Epoch: 10, train/loss: 0.1472, eval/loss: 3.1128, train/auroc: 0.9296, eval/auroc: 0.4887, 


Epoch: 11/100:   0%|          | 0/250 [00:00<?, ?it/s]

[neptune] [info   ] Shutting down background jobs, please wait a moment...
[neptune] [info   ] Done!
[neptune] [info   ] Waiting for the remaining 346 operations to synchronize with Neptune. Do not kill this process.
[neptune] [info   ] All 346 operations synced, thanks for waiting!
[neptune] [info   ] Explore the metadata in the Neptune app: https://app.neptune.ai/arbaaz/bitgrit-air-pollution-1/e/BIT-118/metadata


KeyboardInterrupt: 